<a href="https://colab.research.google.com/github/fatimazain118/C-Language/blob/main/Stacking%20%40Decision%20Tree%20Classifier(Meta%20Learner)/Stacking_freeplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
Zain = pd.read_csv("/content/gdrive/MyDrive/Datasets/Edited-freeplane 1.7.9 - freeplane 1.7.9.csv")

In [29]:
Zain.head(5)

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,6,0,12,155,1,70,32,NO
1,9,0,5,118,1,50,22,NO
2,3,0,2,20,2,0,4,NO
3,2,0,1,7,2,0,1,NO
4,0,0,24,128,2,0,27,NO


In [30]:
Zain['change'] = Zain['change'].map({'YES': 1, 'NO': 0})

In [31]:
Zain.shape

(1278, 8)

In [32]:
Zain.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,6,0,12,155,1,70,32,0
1,9,0,5,118,1,50,22,0
2,3,0,2,20,2,0,4,0
3,2,0,1,7,2,0,1,0
4,0,0,24,128,2,0,27,0


In [33]:
X = Zain.drop('change', axis=1)
y = Zain['change']

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [35]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = DecisionTreeClassifier()
  #level1 = DecisionTreeClassifier()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [36]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [37]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [38]:
models = get_models()

In [39]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.098 (0.297)


In [40]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0]


In [41]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[220  21]
 [ 11   4]]


In [42]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.5897648686030428


In [43]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.14204127163582428


In [44]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5897648686030429


In [45]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.493386412605622


In [46]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93       241
           1       0.16      0.27      0.20        15

    accuracy                           0.88       256
   macro avg       0.56      0.59      0.57       256
weighted avg       0.91      0.88      0.89       256

